<a href="https://colab.research.google.com/github/TaraRasti/Deep_Learning_F20_Assignments/blob/master/Assignment_6/fashion_mnist_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

#print(tfds.list_builders())
dataset, info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']
#print(info)

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteAGTM6M/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteAGTM6M/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [2]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 32

dataset_train = dataset_train.map(convert_types).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(convert_types).batch(batch_size)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 10, horizontal_flip = True, zoom_range = 0.1)

In [4]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Dropout, Activation, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras import Model

class CNNModel(Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        drop_rate = 0.5
        
        self._layers = ([
            Conv2D(32, 3), # 28, 28, 1 -> 26, 26, 32
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(64, 3), # 26, 26, 32 -> 24, 24, 64
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 24, 24, 64 -> 12, 12, 64
            Conv2D(128, 3), # 12, 12, 64 -> 10, 10, 128
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(256, 3), # 10, 10, 128 -> 8, 8, 256
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 8, 8, 256 -> 4, 4, 256
            Flatten(), # 4, 4, 256 -> 4096
            Dense(256), # 4096 -> 256
            BatchNormalization(),
            Dropout(drop_rate),
            Activation(tf.nn.relu),
            Dense(10, activation = 'softmax') # 256 -> 10                        
        ])                
        
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x
       
    
model = CNNModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [7]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
        
@tf.function
def test_step(image, label):
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)

In [9]:
import time

num_epoch = 10
start_time = time.time()

train_accuracies_with_da = [] 
test_accuracies_with_da = []


for epoch in range(num_epoch):    
    for image, label in dataset_train:
        for _image, _label in datagen.flow(image, label, batch_size = batch_size):
            train_step(_image, _label)
            break
        
    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)

    train_accuracies_with_da.append(train_accuracy.result())
    test_accuracies_with_da.append(test_accuracy.result())
    
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}, spent_time: {} min'
    spent_time = time.time() - start_time
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100, spent_time / 60))

Epoch 1, Loss: 0.3074231445789337, Accuracy: 88.57794189453125, Test Loss: 0.28361982107162476, Test Accuracy: 89.7750015258789, spent_time: 6.168840344746908 min
Epoch 2, Loss: 0.2949499487876892, Accuracy: 89.05078887939453, Test Loss: 0.27816829085350037, Test Accuracy: 90.05571746826172, spent_time: 12.355633668104808 min
Epoch 3, Loss: 0.28443244099617004, Accuracy: 89.44294738769531, Test Loss: 0.272437185049057, Test Accuracy: 90.28375244140625, spent_time: 18.500942134857176 min
Epoch 4, Loss: 0.2751888930797577, Accuracy: 89.77930450439453, Test Loss: 0.26923781633377075, Test Accuracy: 90.4366683959961, spent_time: 24.621862812836966 min
Epoch 5, Loss: 0.2671056091785431, Accuracy: 90.07777404785156, Test Loss: 0.2654118537902832, Test Accuracy: 90.5929946899414, spent_time: 30.756550852457682 min
Epoch 6, Loss: 0.2598138451576233, Accuracy: 90.35372161865234, Test Loss: 0.26211509108543396, Test Accuracy: 90.74817657470703, spent_time: 36.86277855237325 min
Epoch 7, Loss: 0.